In [1]:
from cirq_sic import *

import numpy as np
np.set_printoptions(precision=3, suppress=True)

import bqskit as bq
from bqskit.ext import bqskit_to_cirq, cirq_to_bqskit

In [3]:
n = 2
d = 2**n
phi = load_sic_fiducial(d)
ket = rand_ket(d)

qubits = cirq.GridQubit.rect(2, n, top=5, left=1)
system_qubits, ancilla_qubits = qubits[:n], qubits[n:]
prepare_fiducial = ansatz_circuit(phi)
prepare_system = ansatz_circuit(ket)
circuit = cirq.Circuit((prepare_system(system_qubits),\
                        simple_wh_povm(system_qubits, ancilla_qubits, prepare_fiducial=prepare_fiducial, measure=True)))

In [4]:
processor_id = "willow_pink"
device = cirq_google.engine.create_device_from_processor_id(processor_id)
gateset = device.metadata.compilation_target_gatesets[0]
connectivity_graph = device.metadata.nx_graph
#[g.gate for g in gateset.gates]

In [5]:
willow_gateset = [bq.ir.gates.PhasedXZGate(), # cirq.ops.phased_x_z_gate.PhasedXZGate
                  bq.ir.gates.RXGate(), # cirq.ops.phased_x_gate.PhasedXPowGate
                  bq.ir.gates.IdentityGate(), # cirq.I
                  #bq.ir.gates.MeasurementPlaceholder(), #cirq.ops.measurement_gate.MeasurementGate
                  bq.ir.gates.CZGate(), # cirq.CZ
                  #bq.ir.gates.BarrierPlaceholder(), # cirq.ops.wait_gate.WaitGate
                  bq.ir.gates.HGate(), # cirq.ops.common_gates.HPowGate
                  bq.ir.gates.RYGate(), # cirq.ops.common_gates.YPowGate
                  bq.ir.gates.Reset(), #  cirq.ops.common_channels.ResetChannel
                  bq.ir.gates.SGate(), # cirq.ops.clifford_gate.SingleQubitCliffordGate
                  bq.ir.gates.XGate(), # cirq.ops.clifford_gate.SingleQubitCliffordGate
                  bq.ir.gates.YGate(), # cirq.ops.clifford_gate.SingleQubitCliffordGate
                  bq.ir.gates.ZGate(), # cirq.ops.clifford_gate.SingleQubitCliffordGate  
                  bq.ir.gates.RZGate(), # cirq.ops.common_gates.ZPowGate
                  #cirq.ops.global_phase_op.GlobalPhaseGate
                  #bq.ir.gates.RXGate() # cirq.ops.common_gates.XPowGate
]

In [6]:
index_qubit = dict([(i, q) for i, q in enumerate(device.metadata.qubit_set)])
qubit_index = dict([(q, i) for i, q in enumerate(device.metadata.qubit_set)])
qubit_pairs = list(device.metadata.nx_graph.edges)
full_coupling_graph = [(qubit_index[a], qubit_index[b]) for a, b in qubit_pairs]
used_indices = [qubit_index[qubit] for qubit in qubits]
restricted_coupling_graph = [pair for pair in full_coupling_graph if pair[0] in used_indices and pair[1] in used_indices]
coupling_graph = [(used_indices.index(pair[0]), used_indices.index(pair[1])) for pair in restricted_coupling_graph]

In [7]:
bq_circuit = cirq_to_bqskit(circuit)
model = bq.MachineModel(bq_circuit.num_qudits, gate_set=willow_gateset, coupling_graph=coupling_graph)
compiled_bq_circuit, initial_mapping, final_mapping = bq.compile(bq_circuit, model=model, optimization_level=1, with_mapping=True)

In [70]:
compiled_circuit = bqskit_to_cirq(compiled_bq_circuit)
named_qubits = [cirq.NamedQubit("q_%d" % i) for i in range(len(qubits))]
qubit_map = dict([(nq, qubits[i]) for i, nq in enumerate(named_qubits)])

In [33]:
#finished_circuit = compiled_circuit.transform_qubits(qubit_map)
#device.validate_circuit(finished_circuit)

In [47]:
finished_circuit = cirq.drop_negligible_operations(\
                   cirq.drop_terminal_measurements(\
                        compiled_circuit.transform_qubits(qubit_map)))
final_measurement = None
for op in circuit.all_operations():
    if isinstance(op.gate, cirq.MeasurementGate):
        final_measurement = op
        break 
finished_circuit.append(final_measurement)
device.validate_circuit(finished_circuit)

In [48]:
get_gate_counts(finished_circuit)

--- Gate Counts (by type) ---
Rz: 123
Rx: 84
CZPowGate: 19
MeasurementGate: 1


In [49]:
alternate = process_circuit(circuit, connectivity_graph, gateset, qubits)
device.validate_circuit(alternate)
get_gate_counts(alternate)

--- Gate Counts (by type) ---
PhasedXPowGate: 31
CZPowGate: 21
ZPowGate: 4
PhasedXZGate: 13
MeasurementGate: 1


In [64]:
result = cirq.Simulator().simulate(cirq.drop_terminal_measurements(finished_circuit),\
                                   qubit_order=target_order)
abs(result.final_state_vector)**2

array([0.007, 0.117, 0.108, 0.025, 0.026, 0.098, 0.135, 0.086, 0.043,
       0.08 , 0.016, 0.002, 0.004, 0.141, 0.015, 0.097], dtype=float32)

In [78]:
N = 100000
s = cirq.Simulator()
results = s.run(finished_circuit, repetitions=N)
permuted = results.measurements["result"][:, final_mapping]
n_outcomes = 2**permuted.shape[1]
int_outcomes = [big_endian_bits_to_int(bits) for bits in permuted]
counts = collections.Counter(int_outcomes)
for i in range(n_outcomes):
    if i not in counts:
        counts[i] = 0
freqs = np.array([v for k, v in sorted(counts.items())])/counts.total()
freqs

array([0.007, 0.118, 0.107, 0.025, 0.026, 0.097, 0.135, 0.086, 0.045,
       0.08 , 0.016, 0.003, 0.004, 0.141, 0.015, 0.097])

In [79]:
E = wh_povm(phi)
p = np.array([ket.conj() @ e @ ket for e in E]).real; change_conjugate_convention(p)

array([0.007, 0.117, 0.108, 0.025, 0.026, 0.098, 0.135, 0.086, 0.043,
       0.08 , 0.016, 0.002, 0.004, 0.141, 0.015, 0.097])

In [ ]:
#RandomDevicePlacer
# Now synthesize with a cached Compiler
#CIRCUITS = "../experiments/circuits/"
#with open(CIRCUITS+"input.qasm", "w") as f:
#    f.write(cirq.qasm(circ)) # args=cirq.QasmArgs(version="3.0"))